# **Potential Talents**

# Introduction

As a talent sourcing and management company, we are interested in finding talented individuals for sourcing these candidates to technology companies. Finding talented candidates is not easy, for several reasons. The first reason is one needs to understand what the role is very well to fill in that spot, this requires understanding the client’s needs and what they are looking for in a potential candidate. The second reason is one needs to understand what makes a candidate shine for the role we are in search for. Third, where to find talented individuals is another challenge.

The nature of our job requires a lot of human labor and is full of manual operations. Towards automating this process we want to build a better approach that could save us time and finally help us spot potential candidates that could fit the roles we are in search for. Moreover, going beyond that for a specific role we want to fill in we are interested in developing a machine learning powered pipeline that could spot talented individuals, and rank them based on their fitness.

We are right now semi-automatically sourcing a few candidates, therefore the sourcing part is not a concern at this time but we expect to first determine best matching candidates based on how fit these candidates are for a given role. We generally make these searches based on some keywords such as “full-stack software engineer”, “engineering manager” or “aspiring human resources” based on the role we are trying to fill in. These keywords might change, and you can expect that specific keywords will be provided to you.

Assuming that we were able to list and rank fitting candidates, we then employ a review procedure, as each candidate needs to be reviewed and then determined how good a fit they are through manual inspection. This procedure is done manually and at the end of this manual review, we might choose not the first fitting candidate in the list but maybe the 7th candidate in the list. If that happens, we are interested in being able to re-rank the previous list based on this information. This supervisory signal is going to be supplied by starring the 7th candidate in the list. Starring one candidate actually sets this candidate as an ideal candidate for the given role. Then, we expect the list to be re-ranked each time a candidate is starred.

# Data Description:

The data comes from our sourcing efforts. We removed any field that could directly reveal personal details and gave a unique identifier for each candidate.

Attributes:
id : unique identifier for candidate (numeric)

job_title : job title for candidate (text)

location : geographical location for candidate (text)

connections: number of connections candidate has, 500+ means over 500 (text)

Output (desired target):
fit - how fit the candidate is for the role? (numeric, probability between 0-1)

Keywords: “Aspiring human resources” or “seeking human resources”

# Goal(s):

Predict how fit the candidate is based on their available information (variable fit)

# Success Metric(s):

Rank candidates based on a fitness score.

Re-rank candidates when a candidate is starred.

# Current Challenges:

We are interested in a robust algorithm, tell us how your solution works and show us how your ranking gets better with each starring action.

How can we filter out candidates which in the first place should not be in this list?

Can we determine a cut-off point that would work for other roles without losing high potential candidates?

Do you have any ideas that we should explore so that we can even automate this procedure to prevent human bias?

In [ ]:
# Montar Google Drive
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
import pandas as pd
# Ruta correcta al archivo CSV en Google Drive
path_dbset = '/content/gdrive/MyDrive/Proyectos APZIVA/Potential Talents_Proy 3/potential_talents.csv'

# Leer el archivo CSV usando pd.read_csv()
db = pd.read_csv(path_dbset)

# **VISUALIZATION AND MISSING VALUE TREATMENT**
# VISUALIZACIÓN Y TRATAMIENTO DE VALORES FALTANTES

In [ ]:
# Mostrar las primeras 10 filas
print(db.head(7))


In [ ]:
# verify information of the dataset
db.info()

In [ ]:
# Rows and columns information. resultSet(rows, columns)
db.shape

In [ ]:
import numpy as np

# Replace missing value representations with NaN
db.replace(["?", "N/A", "NA", "null", ""], np.nan, inplace=True)

# Check for missing values (NaN) in absolute count
print("\nMissing values (NaN) per column (absolute count):")
print(db.isnull().sum())

# Check for missing values (NaN) as a percentage
print("\nMissing values (NaN) per column (percentage):")
print((db.isnull().sum() / len(db)) * 100)

# **WORD2VEC IMPLEMENTATION**
# IMPLEMENTACION DE WORD2VEC

## **Preprocessing and tokenization**
## Preprocesamiento y tokenización

In [ ]:
#!pip uninstall -y numpy gensim
#!pip install --upgrade numpy gensim
#!pip uninstall -y gensim
!pip install gensim==4.3.3

In [ ]:
import gensim
from gensim.models import Word2Vec
import spacy

# Cargar el modelo de spaCy
nlp = spacy.load('en_core_web_sm') # spacy es una librería donde esta el modelo preentrenado 'en_core_web_sm'.

# la base de datos db tiene la columna con los job titles
job_titles = db['job_title'].tolist()

# Función para preprocesar los títulos de trabajo
def preprocess_text(title):
    doc = nlp(title.lower())  # Convertir a minúsculas y procesar con spaCy
    tokens = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]  # Filtrar palabras clave
    return tokens

# Aplicar preprocesamiento
tokenized_titles = [preprocess_text(title) for title in job_titles]

# Ver los títulos tokenizados
print(tokenized_titles)

SpaCy es una librería de procesamiento de lenguaje natural (NLP) en Python, y en_core_web_sm es uno de sus modelos preentrenados. Es como que spaCy es el "motor", y el modelo en_core_web_sm es uno de los "combustibles" que puede usar para procesar texto en inglés. Además es una librería en la que podemos hacer la limpieza de los títulos (llevar mayúsculas de las palabras que lo componen a minúsculas).
spaCy = librería base (como scikit-learn, pero para texto).
Modelos como en_core_web_sm = entrenados con muchos textos, para que spaCy pueda:
-Tokenizar (separar palabras)
-Lematizar (reducir palabras a su forma base)
-Detectar entidades nombradas (NER)
-Analizar sintaxis

La función def preprocess_text(title), limpia cada título de trabajo:

title.lower() convierte el texto a minúsculas.

nlp(title.lower()) lo analiza con spaCy (tokeniza, etiqueta, etc.).

El bucle [token.lemma_ for token in doc ...] hace tres cosas:

- Se queda con el lema de cada palabra (forma base, por ejemplo, "driving" -- "drive").

- Filtra solo las palabras que son alfabéticas (token.is_alpha)-- se descartan números o símbolos.

- Excluye las palabras vacías (como "the", "and", "of", etc.) usando not token.is_stop.

Línea --> tokenized_titles = [preprocess_text(title) for title in job_titles]. Aplica la función anterior a cada título de la lista y guarda los resultados en tokenized_titles.
Resultado: una lista de listas, donde cada sublista contiene los términos relevantes lematizados del título.

In [ ]:
# Entrenar modelo Word2Vec
word2vec_model = Word2Vec(sentences=tokenized_titles, vector_size=50, window=5, min_count=2, workers=4)

# Guardar el modelo entrenado
word2vec_model.save("word2vec_job_titles.model")

Entrena un modelo Word2Vec con los títulos de trabajo que ya tokenizaste y preprocesaste con spaCy.

sentences=tokenized_titles: le das las frases como listas de palabras (tokens).

vector_size=50: el modelo va a crear vectores de 50 dimensiones para cada palabra.

window=5: considera un contexto de hasta 5 palabras a cada lado de la palabra central.

min_count=2: solo entrena palabras que aparezcan al menos 2 veces.

workers=4: usa 4 núcleos de CPU (paraleliza el trabajo).



Esto guarda el modelo entrenado en un archivo con el nombre "word2vec_job_titles.model".

.model es simplemente una convención de nombre: el archivo podría llamarse "pepito.w2v" y seguiría funcionando igual.

Internamente, se guarda como un archivo binario que contiene:
-Los vectores entrenados
-La configuración del modelo
-El vocabulario

In [ ]:
#Probamos el modelo, en este caso viendo las palabras más cercanas a developer
# Cargar el modelo entrenado
word2vec_model = Word2Vec.load("word2vec_job_titles.model")

# Palabras más similares a "professional"
print(word2vec_model.wv.most_similar("professional", topn=5))

# Palabras más similares a "human"
print(word2vec_model.wv.most_similar("director", topn=5))

print(word2vec_model.wv["professional"])  # Muestra el vector de 10 dimensiones

.wv es el acceso al vocabulario y a los vectores del modelo Word2Vec.
Viene de word vectors, y se usa para consultar el modelo entrenado sin modificarlo.

En el primer y segundo print hay valores entre -1 y 1, que miden la similitud del coseno entre los vectores de dos palabras.
Similitud del coseno = ¿Qué tan alineados están dos vectores en el espacio?

Si el coseno es:
1.0 -- los vectores son exactamente iguales (máxima similitud)
0.0 -- no están relacionados (son ortogonales)
-1.0 -- completamente opuestos (muy raro en lenguaje)

## **Compare titles with query ("human resources manager")**
## Ranqueo de títulos por similitud con la consulta (ej. "Human Resources")


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Función para obtener el vector promedio de un conjunto de palabras
def get_vector(tokens, word2vec_model):
    vectors = [word2vec_model.wv[word] for word in tokens if word in word2vec_model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(word2vec_model.vector_size)

# Generar vectores para cada título de trabajo
job_vectors = np.array([get_vector(tokens, word2vec_model) for tokens in tokenized_titles])

# Tokenizar la consulta
query_tokens = preprocess_text("human resources manager")

# Obtener el vector de la consulta
query_vector = get_vector(query_tokens, word2vec_model)

# Calcular similitud con cada título
similarities = cosine_similarity([query_vector], job_vectors)[0]

# Agregar la similitud al DataFrame original
db["fit_score"] = similarities

# Ordenar candidatos según el puntaje de similitud
# --- SE GENERÓ UNA REINDEXACION (el 0 es el de mayor fit_score), los ID de aquí no coinciden con la BD original ----
db_sorted = db.sort_values(by="fit_score", ascending=False).reset_index(drop=True)
# Mostrar los mejores candidatos
print(db_sorted[["job_title", "fit_score"]].head(50))

## **Evaluation Metrics on the original ranking (similitary with query) - Word2Vec**

In [ ]:
#GENERAMOS LAS ETIQUETAS 0 y 1, que indican si el candidatos es relevante o no
# Agregar la columna relevance con valores nulos
db_sorted["relevance"] = None

# Etiquetar manualmente los primeros 10
manual_labels = [0, 0, 1, 1, 0, 0, 0, 0, 1, 1]  # filas 0 a 9
db_sorted.loc[:9, "relevance"] = manual_labels

# Mostrar los primeros 10 resultados con etiquetas
top_10 = db_sorted.head(10)[["job_title", "fit_score", "relevance"]]
print(top_10)

db_sorted: es el DataFrame ordenado por el fit_score.

db_sorted.loc[:9, "relevance"]:
selecciona las filas de la 0 a la 9 (inclusive) y la columna "relevance".
Es decir: estás apuntando a los primeros 10 registros del ranking.

= manual_labels:
asignamos la lista [0, 0, 1, 1, 0, 0, 0, 0, 1, 1] a esas 10 filas en la columna "relevance".

In [ ]:
#DEFINIR LAS FUNCIONES DE METRICAS
import numpy as np

def precision_at_k(relevance, k):
    """
    Calcula Precision@K.
    `relevance` es una lista o array con etiquetas binarias (0 o 1).
    """
    relevance_at_k = relevance[:k]
    return np.sum(relevance_at_k) / k #suma los 1 en los primeros k elementos de relevance y los divide por k (en este caso 10).
    #Representa la proporción de resultados relevantes en el top-k.

def dcg_at_k(relevance, k):
    """
    Calcula DCG@K (Discounted Cumulative Gain).
    """
    relevance = np.asarray(relevance)[:k]
    return np.sum((2**relevance - 1) / np.log2(np.arange(2, k + 2)))

def ndcg_at_k(relevance, k):
    """
    Calcula nDCG@K normalizando contra el DCG ideal.
    """
    dcg = dcg_at_k(relevance, k)
    ideal_relevance = sorted(relevance, reverse=True)
    idcg = dcg_at_k(ideal_relevance, k)
    return dcg / idcg if idcg > 0 else 0.0

- **Discounted Cumulative Gain (DCG)** mide la utilidad de los documentos
relevantes en un ranking, penalizando aquellos que aparecen más abajo. Cuanto más alto esté un resultado relevante, mayor su contribución al puntaje. El logaritmo penaliza la posición: a mayor profundidad en el ranking, menos valor aporta un resultado relevante.
La función np.asarray(relevance) convierte relevance en un array de NumPy (si aún no lo es). Esto permite aplicar operaciones vectorizadas como la exponenciación y división en el cálculo del DCG.

- **Normalized Discounted Cumulative Gain at k (posición k)nDCG@k** evalúa qué tan bueno es un ranking comparado con el mejor ranking posible (ideal). Es decir, normaliza el valor de DCG dividiéndolo por el IDCG (Ideal DCG), que es el DCG obtenido si los ítems relevantes estuvieran perfectamente ordenados arriba. Si el ranking coloca bien los más relevantes arriba, nDCG@k estará cerca de 1.0. Si el ranking es aleatorio o muy malo, nDCG@k estará cerca de 0.0.

In [ ]:
#EXTRAER LA COLUMNA DE ETIQUETAS MANUALES

# Convertir la columna relevance a entero (por si quedaron como None o strings)
relevance_labels = db_sorted["relevance"].dropna().astype(int).values[:10]

In [ ]:
#CALCULAR LAS METRICAS
K = 10
prec_k = precision_at_k(relevance_labels, K)
ndcg_k = ndcg_at_k(relevance_labels, K)

print(f"Precision@{K}: {prec_k:.3f}")
print(f"nDCG@{K}: {ndcg_k:.3f}")

## **Las métricas que se obtuvieron indican un rendimiento moderado a bajo del método de embedding Word2Vec para la query "human resources manager":**

**Precision@10**: 0.400 Significa que 4 de los 10 primeros resultados fueron relevantes. Podría considerarse aceptable, pero no ideal si se espera alta precisión para una tarea de recomendación o búsqueda.

**nDCG@10**: 0.594 La ganancia acumulada descontada está lejos de 1.0, lo que indica que los documentos más relevantes no están en las posiciones más altas del ranking.

##**Re-ranking of candidates choosing and ID (starred_candidates)**
##Re-ranquear los candidatos a partir de la elección de candidatos estrella


In [ ]:
# El usuario marcó como relevante el job_title con ID 68 y otros
starred_candidates = [65, 76, 88, 80, 67]

# Obtener embeddings de los candidatos estrella/ genera un array con los ID marcados
starred_embeddings = np.array([job_vectors[id-1] for id in starred_candidates])

# Calcular similitud con los títulos estrella
# Calcula la similitud del coseno entre todos los job titles (job_vectors) y los candidatos estrella (starred_embeddings) y luego saca el promedio
starred_similarities = cosine_similarity(job_vectors, starred_embeddings).mean(axis=1)

# Ajuste de pesos: darle más peso a candidatos similares a los marcados
final_scores = 0.7 * similarities + 0.3 * starred_similarities

# Ordenar nuevamente
re_ranked_candidates = sorted(zip(db['id'], job_titles, final_scores), key=lambda x: x[2], reverse=True)

# Convertir la lista de candidatos re-rankeados en un DataFrame
df_re_ranked = pd.DataFrame(re_ranked_candidates, columns=["id", "job_title", "final_score"])

# Mostrar los resultados ordenados en filas
print(df_re_ranked.head(30))  # Muestra los 30 primeros candidatos re-rankeados


## **Irrelevant candidates filter**
## Filtrado de candidatos irrelevantes

In [ ]:
# Definir umbral de corte (percentil 10)
cutoff = np.percentile(final_scores, 10)

# Filtrar candidatos con puntajes por encima del umbral
filtered_candidates = [c for c in re_ranked_candidates if c[2] >= cutoff]
#c: En cada paso del for, c es una tupla como (id, job_title, final_score) de re_ranked_candidates (lista de tuplas con los candidatos)

print(filtered_candidates)

df_re_ranked_filtered = pd.DataFrame(filtered_candidates, columns=["id", "job_title", "final_score"])

# Mostrar los resultados ordenados en filas
print(df_re_ranked_filtered.head(20))  # Muestra los 30 primeros candidatos re-rankeados

# **GLOVE IMPLEMENTATION**
# IMPLEMENTACION DE GLOVE

## Para adaptar el flujo de trabajo de Word2Vec a GloVe, seguiremos estos pasos:

**Entrenamiento: Con GloVe, no es necesario entrenar un modelo, ya que se tienen vectores preentrenados para las palabras.**

- Descargar embeddings preentrenados de GloVe (más eficiente que entrenarlo con mi dataset que es chico).
- Cargar los embeddings y mapearlos a las palabras de los títulos de trabajo.
- Obtener representaciones vectoriales de los títulos de trabajo.
- Comparar los títulos con una consulta utilizando similitud del coseno.
- Re-ranquear candidatos en función de los candidatos estrella.
- Filtrado de candidatos irrelevantes

## **Glove embbedings load**
## Carga de embbedings de GLOVE


A continuación, se descarga e instala el modelo de spaCy "en_core_web_md", que es un modelo de procesamiento de lenguaje natural en inglés que incluye vectores de palabras preentrenados con GloVe.

In [ ]:
# Cargar modelo de spaCy con vectores preentrenados de GloVe
!python -m spacy download en_core_web_md
#cargamos el modelo en memoria
spacy.load("en_core_web_md")

import spacy
#nlp_glove almacena el modelo de spaCy con GloVe, sin interferir con otras variables que puedas haber definido antes (como en Word2Vec)
nlp_glove = spacy.load("en_core_web_md")
print("Modelo cargado correctamente")

## **Text processing with GLOVE** (text or sentence)
## Procesar texto con GLOVE (prueba con un texto o frase cualquiera)

In [ ]:
text = "Machine learning is amazing."
doc_gl = nlp_glove(text)

for token in doc_gl:
    print(f"Palabra: {token.text}, Vector: {token.vector[:5]}")

## **Pre-processing and tokenization**
## Preprocesamiento y tokenización

In [ ]:
# Función para preprocesar los títulos de trabajo
def preprocess_text_glove(title):
    doc_glove = nlp_glove(title.lower())  # Convertir a minúsculas y procesar con spaCy (modelo GloVe)
    tokens = [token.lemma_ for token in doc_glove if token.is_alpha and not token.is_stop]  # Filtrar palabras clave
    return tokens

# Aplicar preprocesamiento con GloVe
tokenized_titles_glove = [preprocess_text_glove(title) for title in job_titles]

# Ver los títulos tokenizados
print(tokenized_titles_glove)

1. Función preprocess_text_glove(title):
title es simplemente una referencia a cada elemento de la lista job_titles mientras se recorre. ¿De dónde sale el título? Cada title proviene de la lista job_titles, que a su vez proviene de la base de datos db['job_title'].

2. Aplicación a la lista job_titles:
Luego, en el paso siguiente, preprocess_text_glove se aplica a todos los títulos de trabajo que están en la lista job_titles. Es decir, la función es iterada sobre cada título de la lista y su resultado (la lista de tokens lematizados) es almacenado en una nueva lista llamada tokenized_titles_glove.

- **title** es el nombre de la variable que usamos para representar un único título de trabajo dentro de la lista job_titles.
- **job_titles** es la lista completa de títulos de trabajo, y cada vez que la iteramos con el código for title in job_titles, tomamos un solo título de la lista en cada iteración.
- El código **preprocess_text_glove(title)** es llamado para cada title (un título de trabajo individual), y la lista **tokenized_titles_glove** almacena los resultados de la tokenización de todos los títulos.


## **Model word vector**
## Modelo de Vectores de palabras

In [ ]:
# Función para obtener el vector de una palabra usando el modelo GloVe
def get_glove_vector(word, model_glove):
    if model_glove.vocab.has_vector(word):  # Verifica si la palabra tiene un vector
        return model_glove(word).vector  # Devuelve el vector
    else:
        return np.zeros(model_glove.vector_length)  # Devuelve un vector de ceros si no está en el vocabulario

# Ver el vector de alguna palabra
print(get_glove_vector("resources", nlp_glove))

## **Compare titles with query ("human resources manager")**
## Ranqueo de títulos por similitud con la consulta ("human resources manager")

In [ ]:
# Función para obtener el vector promedio de un conjunto de palabras usando GloVe
def get_vector_glove(tokens, model_glove):
    vectors_glove = [model_glove(token).vector for token in tokens if model_glove(token).vector.any()]
    return np.mean(vectors_glove, axis=0) if vectors_glove else np.zeros(model_glove.vector_length)

# Generar vectores para cada título de trabajo
job_vectors_glove = np.array([get_vector_glove(tokens, nlp_glove) for tokens in tokenized_titles_glove])

# Tokenizar la consulta
query_tokens_glove = preprocess_text_glove("human resources manager")

# Obtener el vector de la consulta
query_vector_glove = get_vector_glove(query_tokens_glove, nlp_glove)

# Calcular similitud con cada título
similarities_glove = cosine_similarity([query_vector_glove], job_vectors_glove)[0]

# Agregar la similitud al DataFrame original
db["fit_score"] = similarities_glove

# Ordenar candidatos según el puntaje de similitud
# --- SE GENERÓ UNA REINDEXACION (el 0 es el de mayor fit_score), los ID de aquí no coinciden con la BD original ----
db_sorted_glove = db.sort_values(by="fit_score", ascending=False).reset_index(drop=True)

# Mostrar los mejores candidatos
print(db_sorted_glove[["job_title", "fit_score"]].head(50))

## **Evaluation Metrics on the original ranking (similitary with query) - GLOVE**

In [ ]:
# GENERAMOS LAS ETIQUETAS 0 y 1, que indican si el candidato es relevante o no
# Agregar la columna relevance con valores nulos en el DataFrame específico de Glove
db_sorted_glove["relevance"] = None

# Etiquetar manualmente los primeros 10
manual_labels_glove = [0, 1, 1, 1, 0, 1, 0, 0, 1, 0]  # filas 0 a 9
db_sorted_glove.loc[:9, "relevance"] = manual_labels_glove

# Mostrar los primeros 10 resultados con etiquetas
top_10_glove = db_sorted_glove.head(10)[["job_title", "fit_score", "relevance"]]
print(top_10_glove)

In [ ]:
# OBTENER LOS LABELS DESDE db_sorted_glove
relevance_labels_glove = db_sorted_glove["relevance"].head(10).astype(int).tolist()

# DEFINIR K
K = 10

# CALCULAR LAS MÉTRICAS PARA GLOVE
prec_k_glove = precision_at_k(relevance_labels_glove, K)
ndcg_k_glove = ndcg_at_k(relevance_labels_glove, K)

# MOSTRAR RESULTADOS
print(f"Precision@{K} (Glove): {prec_k_glove:.3f}")
print(f"nDCG@{K} (Glove): {ndcg_k_glove:.3f}")

**Notas importantes:**

- No hace falta redefinir las funciones precision_at_k, dcg_at_k, ni ndcg_at_k porque ya fueron definidas en la sección de WORD2VEC.

- Asegurate de que la columna "relevance" de db_sorted_glove contenga los 10 valores manuales ya asignados.

- Este código no pisa nada de Word2Vec porque usa nuevas variables (db_sorted_glove, relevance_labels_glove, etc.).

##**CONCLUSION - Evaluación de métricas.**

nDCG@10 de 0.753 es una métrica bastante alta para Glove, y efectivamente, es más alta que la obtenida para Word2Vec (que fue 0.594).**

##**Re-ranking of candidates choosing and ID (starred_candidates)**
##Re-ranquear los candidatos a partir de la elección de candidatos estrella

In [ ]:
# El usuario marcó como relevante el job_title con ID 68 y otros
starred_candidates_glove = [68, 69, 80, 87, 88]

# Obtener embeddings de los candidatos estrella con GloVe
starred_embeddings_glove = np.array([job_vectors_glove[id-1] for id in starred_candidates_glove])

# Calcular similitud con los títulos estrella
starred_similarities_glove = cosine_similarity(job_vectors_glove, starred_embeddings_glove).mean(axis=1)

# Ajuste de pesos: darle más peso a candidatos similares a los marcados
final_scores_glove = 0.7 * similarities_glove + 0.3 * starred_similarities_glove

# Ordenar nuevamente
re_ranked_candidates_glove = sorted(zip(db['id'], job_titles, final_scores_glove), key=lambda x: x[2], reverse=True)

# Convertir la lista de candidatos re-rankeados en un DataFrame
df_re_ranked_glove = pd.DataFrame(re_ranked_candidates_glove, columns=["id", "job_title", "final_score"])

# Mostrar los resultados ordenados en filas
print(df_re_ranked_glove.head(30))  # Muestra los 30 primeros candidatos re-rankeados

## **Irrelevant candidates filter**
## Filtrado de candidatos irrelevantes

In [ ]:
# Definir umbral de corte (percentil 10)
cutoff_glove = np.percentile(final_scores_glove, 10)

# Filtrar candidatos con puntajes por encima del umbral
filtered_candidates_glove = [c for c in re_ranked_candidates_glove if c[2] >= cutoff_glove]

# Convertir la lista de candidatos filtrados en un DataFrame
df_re_ranked_filtered_glove = pd.DataFrame(filtered_candidates_glove, columns=["id", "job_title", "final_score"])

# Mostrar los resultados ordenados en filas
print(df_re_ranked_filtered_glove.head(20))  # Muestra los 20 primeros candidatos filtrados

# **FAST TEXT IMPLEMENTATION**
# IMPLEMENTACION DE FAST TEXT

In [ ]:
from gensim.models import FastText

# Entrenar el modelo FastText
fasttext_model = FastText(sentences=tokenized_titles, vector_size=50, window=5, min_count=2, workers=4)

# Guardar el modelo entrenado
fasttext_model.save("fasttext_job_titles.model")

- sentences=tokenized_titles: es la lista de tokens para cada Job Title de la base. Se hizo antes de implementar Word2Vec.

- vector_size=50: igual que en Word2Vec.

- La ventaja principal de FastText sobre Word2Vec es que puede generar vectores para palabras no vistas, gracias a su representación basada en sub-palabras.

In [ ]:
# INSPECCIONAR EL MODELO

# Cargar el modelo entrenado
fasttext_model = FastText.load("fasttext_job_titles.model")

# Palabras más similares a "professional"
print(fasttext_model.wv.most_similar("professional", topn=5))

# Palabras más similares a "director"
print(fasttext_model.wv.most_similar("director", topn=5))

# Vector del término "professional"
print(fasttext_model.wv["professional"])

- Cargar el modelo entrenado.

- Explorar similitud entre palabras (primero professional y luego director).

- Obtener el vector de una palabra específica (professional).

## **Compare titles with query ("Human Resources")**
## Ranqueo de títulos por similitud con la consulta (ej. "Human Resources")

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Función para obtener el vector promedio de un conjunto de palabras
def get_vector_fasttext(tokens, fasttext_model):
    vectors_fasttext = [fasttext_model.wv[word] for word in tokens if word in fasttext_model.wv]
    return np.mean(vectors_fasttext, axis=0) if vectors_fasttext else np.zeros(fasttext_model.vector_size)

# Generar vectores para cada título de trabajo
job_vectors_fasttext = np.array([get_vector_fasttext(tokens, fasttext_model) for tokens in tokenized_titles])

# Tokenizar y vectorizar la consulta
query_tokens_fasttext = preprocess_text("Human Resources")
query_vector_fasttext = get_vector_fasttext(query_tokens_fasttext, fasttext_model)

# Calcular similitud con cada título
similarities_fasttext = cosine_similarity([query_vector_fasttext], job_vectors_fasttext)[0]

print(len(similarities_fasttext), len(db))

# Crear copia del DataFrame y agregar la similitud
db_fasttext = db.copy()
db_fasttext["fit_score_fasttext"] = similarities_fasttext

# Ordenar candidatos según el puntaje de similitud
db_sorted_fasttext = db_fasttext.sort_values(by="fit_score_fasttext", ascending=False)

# Mostrar los mejores candidatos
print(db_sorted_fasttext[["job_title", "fit_score_fasttext"]].head(40))

## **Simple Quantitative Analysis - Comparison between Word2Vec and FastText**
## Análisis cuantitativo simple - comparación Word2Vec vs. Fast Text.

## **Show Top 10 Results: Word2Vec vs. FastText**
## Mostrar los Top 10 resultados de Word2Vec vs. FastText

In [ ]:
# Mostrar los 10 títulos con mayor similitud usando Word2Vec
print("Top 10 resultados con Word2Vec:")
print(db_sorted[["job_title", "fit_score"]].head(10))

# Mostrar los 10 títulos con mayor similitud usando FastText
print("\nTop 10 resultados con FastText:")
print(db_sorted_fasttext[["job_title", "fit_score_fasttext"]].head(10))

In [ ]:
print([col for col in db_fasttext.columns])
print(len(db_fasttext["fit_score"]))
print(len(db_fasttext["fit_score_fasttext"]))

## **Calculate the Correlation Between Similarity Scores**
## Calcular la correlación entre los puntajes de similitud

In [ ]:
from scipy.stats import pearsonr

# Calcular la correlación de Pearson entre los scores de ambos modelos
correlacion, _ = pearsonr(db_fasttext["fit_score"], db_fasttext["fit_score_fasttext"])
print(f"Correlación entre Word2Vec y FastText: {correlacion:.3f}")

- ¿Qué significa una correlación de 0.884?
La correlación de Pearson mide la relación lineal entre dos conjuntos de valores. Va de -1 (correlación negativa perfecta) a +1 (positiva perfecta).

- En mi caso:
fit_score: similitud de Word2Vec entre la consulta y los títulos.
fit_score_fasttext: similitud de FastText entre la misma consulta y los mismos títulos.

- Un valor de 0.884 significa que:
Ambos modelos están dando resultados muy parecidos en términos de similitud. Cuando uno da un puntaje alto, el otro también tiende a hacerlo.

## **Re-ranking of candidates choosing and ID (starred_candidates)**
## Re-ranquear los candidatos a partir de la elección de candidatos estrella*

In [ ]:
# Step 1: Select indexes of starred candidates (chosen manually or from top scores)
starred_candidate_indices_fasttext = [64, 87, 98]  #

# Step 2: Get FastText vectors for those starred candidates
starred_vectors_fasttext = job_vectors_fasttext[starred_candidate_indices_fasttext]

# Step 3: Compute the average vector of the starred candidates
starred_avg_vector_fasttext = np.mean(starred_vectors_fasttext, axis=0)

# Step 4: Compute cosine similarity between all job vectors and the starred average
re_rank_similarities_fasttext = cosine_similarity([starred_avg_vector_fasttext], job_vectors_fasttext)[0]

# Step 5: Create a new DataFrame to avoid overwriting
db_reranked_fasttext = db.copy()
db_reranked_fasttext["fit_score_reranked_fasttext"] = re_rank_similarities_fasttext

# Reasignar la columna 'fit_score_fasttext' desde 'db_fasttext' a 'db_reranked_fasttext'
db_reranked_fasttext["fit_score_fasttext"] = db_fasttext["fit_score_fasttext"]

# Ahora que tenemos la columna 'fit_score_fasttext', podemos ordenarlo
db_sorted_reranked_fasttext = db_reranked_fasttext.sort_values(by="fit_score_reranked_fasttext", ascending=False)

# Mostrar los mejores candidatos
print(db_sorted_reranked_fasttext[["job_title", "fit_score_fasttext", "fit_score_reranked_fasttext"]].head(20))

# **S-BERT IMPLEMENTATION**
# IMPLEMENTACIÓN CON S-BERT

In [ ]:
#CODE 1 - Load S-BERT model (not necesary to train from skratch)
#CÓDIGO 1 - Cargar S-BERT (no es necesario entrenarlo desde cero)

# Instalar la librería si no la tienes
# !pip install sentence-transformers

from sentence_transformers import SentenceTransformer

# Load a pretrained S-BERT model
# Cargar un modelo S-BERT preentrenado
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')  # Modelo rápido y muy efectivo

# (Opcional) Guardarlo localmente si quieres
sbert_model.save('sbert_job_titles_model')

The model all-MiniLM-L6-v2 is very common for production: is light, fast and
precisely for embeddings of short sentences as job titles.

The model all-mpnet-base-v2 is more robust (but more heavy).

In [ ]:
# CODE 2 - S-BERT model test
# CÓDIGO 2 - Prueba del modelo S-BERT

from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Example job titles to test
# Ejemplos de títulos para probar
example_titles = ["human resources manager", "human resources director", "software engineer", "talent acquisition specialist", "director of engineering"]

# Encode the job titles to get their embeddings
# Obtener los embeddings de los títulos
example_embeddings_SBert = sbert_model.encode(example_titles, normalize_embeddings=True)

# Calculate cosine similarity between all pairs
# Calcular la similitud de coseno entre todos los pares
similarity_matrix_SBert = cosine_similarity(example_embeddings_SBert)

# Show the similarity matrix
# Mostrar la matriz de similitudes
print("Similarity matrix between example titles:")
print("Matriz de similitud entre ejemplos:")
print(similarity_matrix_SBert)

# Select the embedding of the query title directly from precomputed embeddings
# Seleccionar el embedding de la consulta directamente de los embeddings precomputados
query = "human resources manager"
query_idx = example_titles.index(query)
query_embedding_SBert = example_embeddings_SBert[query_idx].reshape(1, -1)

# Calculate similarity between the query and all example titles
# Calcular la similitud entre la consulta y todos los títulos de ejemplo
similarities_SBert = cosine_similarity(query_embedding_SBert, example_embeddings_SBert)[0]

# Sort titles by similarity score
# Ordenar los títulos por puntaje de similitud
sorted_indices = np.argsort(similarities_SBert)[::-1]  # From highest to lowest / De mayor a menor

# Display most similar titles
# Mostrar los títulos más similares
print("\nMost similar titles to 'human resources manager':")
print("\nTítulos más similares a 'human resources manager':")
for idx in sorted_indices:
    print(f"{example_titles[idx]}: {similarities_SBert[idx]:.4f}")

**Word2Vec** trabaja a nivel de palabra. Cada palabra ("professional", "director") tiene su propio vector individual. El modelo está pensado para aprender relaciones entre palabras basadas en su contexto local (ventana de palabras cercanas).

**S-BERT**, en cambio, trabaja a nivel de frases completas ("oraciones", "títulos de trabajo", "párrafos cortos").
No tiene vectores individuales por palabra, sino que genera un embedding para toda la oración o frase.
- Por eso se tomaron frases como "human resources manager", "software engineer", etc., y se generó un solo vector por frase.

En la matriz de similitud

Cada fila representa un título de trabajo (embedding).

Cada columna representa también un título de trabajo (embedding).

El valor en la posición (i, j) de la matriz es la similitud del coseno entre el título i y el título j.

La diagonal principal (cuando i == j) es la comparación de un título consigo mismo, siempre es 1.

**sbert_model.encode** genera resultados aleatorios pequeños si no se setea **normalize_embeddings=True**.
Esto significa que los vectores no están normalizados y, por tanto, su similitud consigo mismos no es exactamente 1. Luego de normalizar, la similitud de un vector consigo mismo se convirtió en 1.

In [ ]:
db.info()


## **Compare titles with query ("human resources manager")**
## Ranqueo de títulos por similitud con la consulta (ej. "human resources manager")

In [ ]:
# CODE 3 - Ranking job titles by similarity to the query using S-BERT
# CÓDIGO 3 - Ranqueo de títulos de trabajo por similitud a la consulta usando S-BERT

# Function to preprocess and encode job titles
# Función para preprocesar y codificar títulos de trabajo
def encode_titles_SBert(titles_list):
    return sbert_model.encode(titles_list, normalize_embeddings=True)

# Generate embeddings for all job titles in the database
# Generar embeddings para todos los títulos en la base de datos
job_titles_SBert = db['job_title'].tolist()
job_embeddings_SBert = encode_titles_SBert(job_titles_SBert)

# Preprocess and encode the query
# Preprocesar y codificar la consulta
query_SBert = "human resources manager"
query_embedding_SBert = sbert_model.encode([query_SBert], normalize_embeddings=True)

# Calculate cosine similarity between the query and all job titles
# Calcular la similitud de coseno entre la consulta y todos los títulos
similarities_SBert = cosine_similarity(query_embedding_SBert, job_embeddings_SBert)[0]

# Add the similarity scores to the original DataFrame
# Agregar los puntajes de similitud al DataFrame original
db["fit_score_SBert"] = similarities_SBert

# Sort candidates based on the fit score
# Ordenar candidatos basados en el puntaje de ajuste
db_sorted_SBert = db.sort_values(by="fit_score_SBert", ascending=False).reset_index(drop=True)

# Display the top candidates
# Mostrar los mejores candidatos
print(db_sorted_SBert[["job_title", "fit_score_SBert"]].head(50))

- Primero definimos una función encode_titles_SBert para codificar listas de títulos usando S-BERT, con normalización.

- Generamos los embeddings para todos los job titles que tenés en tu base de datos.

- Codificamos el query "Human Resources" usando también normalize_embeddings=True.

- Calculamos la similaridad del coseno entre la consulta y cada job title.

- Guardamos los resultados en una nueva columna llamada "fit_score_SBert" en tu DataFrame.

- Finalmente ordenamos de mayor a menor similitud y mostramos los 50 mejores candidatos (hay reindexación de los ID, el 0 es el de mayor fit).

**Algunos puntos importantes:**

**Primero** (1):

titles_list: nombre interno del parámetro

job_titles_SBert: la lista real que le pasamos cuando llamamos a la función

**Segundo** (2):

¿Qué hace .tolist()?
 .tolist() es un método de pandas que convierte una columna (que es un objeto especial de tipo Series) en una lista de Python común y corriente. Lo hacemos porque S-BERT espera listas de textos, no una Series de pandas.

**Tercero** (3):

Para codificar muchos títulos (todos los job_titles del DataFrame), usamos la función encode_titles_SBert(), que adentro llama a sbert_model.encode() con normalize_embeddings=True.

Para codificar un solo query ("Human Resources"), como es solo un string y no queremos crear otra función, directamente usamos:
query_embedding_SBert = **sbert_model.encode**([query_SBert], normalize_embeddings=True)

##**Evaluation Metrics on the original ranking (similitary with query) - S-BERT**

In [ ]:
# GENERAMOS LAS ETIQUETAS 0 y 1, que indican si el candidato es relevante o no
# Agregar la columna relevance con valores nulos en el DataFrame específico de S-BERT
db_sorted_SBert["relevance"] = None

# Etiquetar manualmente los primeros 10
manual_labels_SBert = [1, 1, 0, 0, 0, 0, 0, 0, 0, 0]  # filas 0 a 9
db_sorted_SBert.loc[:9, "relevance"] = manual_labels_SBert

# Mostrar los primeros 10 resultados con etiquetas
top_10_SBert = db_sorted_SBert.head(10)[["job_title", "fit_score_SBert", "relevance"]]
print(top_10_SBert)

In [ ]:
# Asegurarse de que los valores en relevance no sean None y convertirlos a int
relevance_labels_SBert = db_sorted_SBert["relevance"].fillna(0).astype(int).tolist()

# Definir K
K = 10

# Calcular métricas
prec_k_SBert = precision_at_k(relevance_labels_SBert, K)
ndcg_k_SBert = ndcg_at_k(relevance_labels_SBert, K)

# Mostrar resultados
print(f"Precision@{K} (SBert): {prec_k_SBert:.3f}")
print(f"nDCG@{K} (SBert): {ndcg_k_SBert:.3f}")

##**CONCLUSIONES - Evaluación de Métricas - S-Bert**

- Un valor de **nDCG@10 = 1.000** significa que el modelo ordenó perfectamente los candidatos según tus etiquetas manuales. Puso primero los más relevantes, luego los menos, sin errores en la jerarquía.

- **Precision@10 = 0.200**: Significa que de los primeros 10 candidatos, solo 2 fueron marcados como relevantes. O sea, el modelo ordenó bien a los pocos que sí eran relevantes, pero el resto del top 10 no fue útil. El modelo puso en primer lugar a los mejores pero la mayoría del top 10 no era relevante según tu criterio.

- Para S-BERT, **"Aspiring Human Resources Specialist"** está semánticamente muy cerca de "Human Resources Manager" -- comparten contexto, área y muchas palabras clave.

**Conclusión conceptual**
nDCG alto, Precision baja: S-BERT es muy bueno entendiendo significado general y ordenando lo que considera relevante, pero no siempre alinea ese juicio con la necesidad específica (e.g., que un "aspirante" no es apto aún).

Esto revela que los embeddings preentrenados no siempre reflejan los criterios personalizados de relevancia.

Requiere ajuste fino o re-entrenamiento en dominios específicos, o combinarlo con filtros adicionales (e.g., excluir títulos con “aspiring”).



##**Re-ranking of candidates choosing and ID (starred_candidates)**
##Re-ranquear los candidatos a partir de la elección de candidatos estrella

In [ ]:
# CODE 4 - Re-ranking of candidates after selecting starred candidates
# CÓDIGO 4 - Re-ranqueo de candidatos después de seleccionar candidatos estrella

# Define the IDs of the starred candidates
# Definir los IDs de los candidatos marcados como estrella
starred_candidates_SBert = [73, 87, 50, 67, 83]

# Get embeddings of the starred candidates
# Obtener los embeddings de los candidatos estrella
starred_embeddings_SBert = np.array([job_embeddings_SBert[id-1] for id in starred_candidates_SBert])

# Calculate similarity between all job titles and the starred candidates
# Calcular la similitud entre todos los títulos de trabajo y los candidatos estrella
starred_similarities_SBert = cosine_similarity(job_embeddings_SBert, starred_embeddings_SBert).mean(axis=1)

# Adjust the final scores: give more weight to candidates similar to the starred ones
# Ajustar los puntajes finales: dar más peso a los candidatos similares a los marcados
final_scores_SBert = 0.7 * similarities_SBert + 0.3 * starred_similarities_SBert

# Re-rank candidates based on the new scores
# Re-ranquear los candidatos basándose en los nuevos puntajes
re_ranked_candidates_SBert = sorted(zip(db['id'], job_titles_SBert, final_scores_SBert), key=lambda x: x[2], reverse=True)

# Convert the re-ranked list into a DataFrame
# Convertir la lista re-ranqueada en un DataFrame
df_re_ranked_SBert = pd.DataFrame(re_ranked_candidates_SBert, columns=["id", "job_title", "final_score"])

# Show the top re-ranked candidates
# Mostrar los principales candidatos re-ranqueados
print(df_re_ranked_SBert.head(30))

**OBSERVACIÓN**

¿Por qué usamos zip(db['id'], job_titles_SBert, final_scores_SBert) y no solo final_scores_SBert? Porque no solo queremos ordenar los puntajes, sino también conservar la información asociada a cada puntaje: el ID del candidato, el título de trabajo (job title), y el puntaje final (final score).

zip() lo que hace es empaquetar los tres elementos juntos en tuplas.

Cada tupla luce así:

(15, "software engineer", 0.8654)

(68, "human resources manager", 0.9432)

## **Irrelevant candidates filter (percentil 10)**
## Filtrado de candidatos irrelevantes (percentil 10)

In [ ]:
# CODE 5 - Candidate filtering based on cutoff
# CÓDIGO 5 - Filtrado de candidatos basado en umbral

# Define a cutoff threshold (e.g., 10th percentile)
# Definir un umbral de corte (por ejemplo, percentil 10)
cutoff_SBert = np.percentile(final_scores_SBert, 10)

# Filter candidates with scores above the threshold
# Filtrar candidatos con puntajes por encima del umbral
filtered_candidates_SBert = [c for c in re_ranked_candidates_SBert if c[2] >= cutoff_SBert]

# Convert the filtered list to a DataFrame
# Convertir la lista filtrada en un DataFrame
df_re_ranked_filtered_SBert = pd.DataFrame(filtered_candidates_SBert, columns=["id", "job_title", "final_score"])

# Show the top filtered candidates
# Mostrar los principales candidatos filtrados
print(df_re_ranked_filtered_SBert.head(20)) # Show top 20 candidates / Mostrar los 20 mejores candidatos